In [1]:
# Warming Up
import pandas as pd
import numpy as np

fPath = './data/vgsales.csv'
df = pd.read_csv(fPath)

# trying to create 1.5% worth of duplicates
rws = df.shape[0]
col = df.shape[1]

'''
# Commented out def, as there are actual duplicates once rank is removed

from random import randrange
rwRtio = int(.015 * rws) # float is truncated
print('Number of instances before duplication = %d' % (rws))

def eenyMeeny():
     # select a random row
    tg = df.iloc[randrange(0, rws)]
    # return randomly selected row
    return tg

for i in range(rwRtio):
    # Also reported issues on this line
    df = df.append(eenyMeeny(), ignore_index=True)
rws = df.shape[0]
'''

print('Number of instances = %d' % (rws))
print('Number of attributes = %d' % (col))
print(df.head())
# Tables that Michael Berbach will use
dfMB = df

Number of instances = 16598
Number of attributes = 11
   Rank                      Name Platform    Year         Genre Publisher  \
0     1                Wii Sports      Wii  2006.0        Sports  Nintendo   
1     2         Super Mario Bros.      NES  1985.0      Platform  Nintendo   
2     3            Mario Kart Wii      Wii  2008.0        Racing  Nintendo   
3     4         Wii Sports Resort      Wii  2009.0        Sports  Nintendo   
4     5  Pokemon Red/Pokemon Blue       GB  1996.0  Role-Playing  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  
0     41.49     29.02      3.77         8.46         82.74  
1     29.08      3.58      6.81         0.77         40.24  
2     15.85     12.88      3.79         3.31         35.82  
3     15.75     11.01      3.28         2.96         33.00  
4     11.27      8.89     10.22         1.00         31.37  


### Missing Values

In [2]:
data = df.replace('?',np.NaN)

print('Number of instances = %d' % (rws))
print('Number of attributes = %d' % (col))

print('Number of missing values:')
for col in df.columns:
    print('\t%s: %d' % (col,df[col].isna().sum()))

Number of instances = 16598
Number of attributes = 11
Number of missing values:
	Rank: 0
	Name: 0
	Platform: 0
	Year: 271
	Genre: 0
	Publisher: 58
	NA_Sales: 0
	EU_Sales: 0
	JP_Sales: 0
	Other_Sales: 0
	Global_Sales: 0


In [3]:
print('Number of rows in original data = %d' % (rws))
df = df.dropna()
print('Number of rows after discarding missing values = %d' % (df.shape[0]))
print('Difference in number of rows = %d' % (rws-df.shape[0]))
# update rws
rws = df.shape[0]

Number of rows in original data = 16598
Number of rows after discarding missing values = 16291
Difference in number of rows = 307


### Splitting Training, Validation, and Testing data
To get our data set ready for a neural network, we need to split the data set into training, validation, and testing data. We will also need to use randomization to try and avoid any disproportionate records or bias. Choosing to split prior to other steps such as normalization because the testing data will be coming in as-is.
Author: Rahul Gupta

#### Shuffling
Using shuffling to randomize the dataset before splitting to avoid having any disproportionate records or bias.

In [4]:
df_shuffled = df.sample(frac=1, random_state=42)

#### Splitting
We'll use splitting to divide the data set into training, validation, and testing. This will be crucial for training the neural network and ensuring it's working as expected before using it on test data. For this scenario, we will use a 70%, 15%, and 15% split for training, validation, and testing accordingly. 

In [5]:
from sklearn.model_selection import train_test_split

train_df, test_valid_df = train_test_split(df_shuffled, test_size=0.3, random_state=42)
test_df, valid_df = train_test_split(test_valid_df, test_size=0.5, random_state=42)

train_df[20:23]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
15974,15977,Misshitsu no Sacrifice,PSP,2010.0,Adventure,D3Publisher,0.00,0.00,0.02,0.00,0.02
9177,9179,Eragon,X360,2006.0,Action,Vivendi Games,0.12,0.01,0.00,0.01,0.14
13655,13657,Phantom Kingdom Portable,PSP,2011.0,Role-Playing,Nippon Ichi Software,0.00,0.00,0.04,0.00,0.04


#### Dropping
Depending on what we want to predict with the neural network, we can drop the appropriate column for the test and validation dataframes. For example, if we want the neural network to predict Global_Sales, we would drop that column for validation and training dataframes.

In [6]:
valid_global_sales = valid_df['Global_Sales']
valid_df = valid_df.drop(['Global_Sales'], axis=1)
test_df = test_df.drop(['Global_Sales'], axis=1)

test_df[20:23]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
9941,9943,CSI: Crime Scene Investigation,XB,2004.0,Adventure,Ubisoft,0.09,0.03,0.00,0.00
4188,4190,Quest 64,N64,1998.0,Role-Playing,Konami Digital Entertainment,0.38,0.08,0.00,0.01
15323,15326,Meikyuu Cross Blood: Reloaded,X360,2011.0,Action,Unknown,0.00,0.00,0.02,0.00


### Outliers
One way of preprocessing the data set is to prepare it for predicting a video game's sales based on its genre. It could be useful for publishers to know what genre of game to develop based on a prediction with this data. To do so, we will need to prepare the data set for normalization and remove outliers.
- Author: Rahul G

#### One Hot Encoding
Start by using one-hot encoding to convert the genre categorical attribute into numerical values. One-hot encoding is crucial if we are including genre and normalizing the data set since categorical attributes without one-hot encoding are not compatible with normalization.

In [7]:
normalized = df
one_hot = pd.get_dummies(normalized['Genre'])

one_hot[20:23]

,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
20,False,False,False,False,False,False,False,True,False,False,False,False
21,False,False,False,False,True,False,False,False,False,False,False,False
22,False,False,False,False,True,False,False,False,False,False,False,False


#### Dropping Columns
We also need to drop columns that do not fit the specific needs of the neural network. In this case, anything that is not genre or sales data. However, we will also drop genre as well and later replace it with the one-hot encoded columns made in the previous step.

In [8]:
print('Number of columns before discarding columns = %d' % (normalized.shape[1]))

normalized = normalized.drop(['Rank', 'Name', 'Platform', 'Genre', 'Year', 'Publisher'], axis=1)

print('Number of columns after discarding columns = %d' % (normalized.shape[1]))

Number of columns before discarding columns = 11
Number of columns after discarding columns = 5


#### Concatenating Rows and Columns
After dropping the unnecessary columns, we now want to add the one-hot encoded genres to the main dataframe. To add the one-hot encoded rows and columns to the dataframe, we will use the pandas function concat, which will combine them both into one dataframe.

In [9]:
normalized = pd.concat([one_hot, normalized], axis=1)

normalized[20:23]

,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
20,False,False,False,False,False,False,False,True,False,False,False,False,6.42,4.52,6.04,1.37,18.36
21,False,False,False,False,True,False,False,False,False,False,False,False,10.83,2.71,4.18,0.42,18.14
22,False,False,False,False,True,False,False,False,False,False,False,False,9.54,3.44,3.84,0.46,17.28


#### Normalization
Now, we will normalize the data to find and remove any outliers. Doing so will make it easier for the neural network to find and identify patterns with genre and sales data. All data will be converted into z scores with the following formula: z score = (x - mean)/standard deviation.

In [10]:
normalized = (normalized-normalized.mean())/normalized.std()

normalized[20:23]

,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
20,-0.499294,-0.291259,-0.232571,-0.339754,-0.238235,-0.190407,-0.285138,3.175168,-0.29225,-0.234325,-0.40585,-0.207095,7.483114,8.584810,19.113686,6.952620,11.368968
21,-0.499294,-0.291259,-0.232571,-0.339754,4.197285,-0.190407,-0.285138,-0.314925,-0.29225,-0.234325,-0.40585,-0.207095,12.845259,5.030933,13.149844,1.954799,11.228603
22,-0.499294,-0.291259,-0.232571,-0.339754,4.197285,-0.190407,-0.285138,-0.314925,-0.29225,-0.234325,-0.40585,-0.207095,11.276740,6.464264,12.059680,2.165234,10.679905


#### Removing Outliers
After getting z scores, it is now possible to find outliers and remove them. Since all columns have been converted into z scores, we need to see if values are greater than -3 or less than or equal to 3. Rows that fall into that range are not outliers and are included in removed_outliers. However, rows that fall outside of that range are outliers and are excluded from the dataframe removed_outliers.


In [11]:
print('Number of rows before discarding outliers = %d' % (normalized.shape[0]))

removed_outliers = normalized.loc[((normalized > -3).sum(axis=1)==17) & ((normalized <= 3).sum(axis=1)==17)]
print('Number of rows after discarding outliers values = %d' % (removed_outliers.shape[0]))

removed_outliers[20:23]

Number of rows before discarding outliers = 16291
Number of rows after discarding outliers values = 7060


,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
325,2.002705,-0.291259,-0.232571,-0.339754,-0.238235,-0.190407,-0.285138,-0.314925,-0.29225,-0.234325,-0.405850,-0.207095,1.889947,2.144635,1.254224,1.060453,2.060230
327,-0.499294,-0.291259,-0.232571,2.943125,-0.238235,-0.190407,-0.285138,-0.314925,-0.29225,-0.234325,-0.405850,-0.207095,2.084492,2.596232,-0.252768,1.428713,2.053849
339,-0.499294,-0.291259,-0.232571,-0.339754,-0.238235,-0.190407,-0.285138,-0.314925,-0.29225,-0.234325,2.463813,-0.207095,2.266878,2.026827,-0.220704,1.586539,1.996428


### Duplicate Data

In [12]:

# It was found that duplicates exist when the rank attribute is removed
dfMB = dfMB.drop(['Rank'], axis=1)
dups = dfMB.duplicated()
print('Number of duplicate rows = %d' % (dups.sum()))
dfMB.loc[dfMB.duplicated(keep=False)]

Number of duplicate rows = 1


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
14997,Wii de Asobu: Metroid Prime,Wii,NaN,Shooter,Nintendo,0.0,0.0,0.02,0.0,0.02
14999,Wii de Asobu: Metroid Prime,Wii,NaN,Shooter,Nintendo,0.0,0.0,0.02,0.0,0.02


In [13]:
print('Number of rows before discarding duplicates = %d' % (rws))
dfMB = dfMB.drop_duplicates()
print('Number of rows after discarding duplicates = %d' % (dfMB.shape[0]))

Number of rows before discarding duplicates = 16291
Number of rows after discarding duplicates = 16597


### Shuffling Dataframes

In [14]:
dfMB = dfMB.reindex(np.random.permutation(dfMB.index))
dfMB.reset_index(inplace=True, drop=True)
dfMB
# Shuffling is aparent as it was
# originally ranked in order of global sales

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Rapala Tournament Fishing!,Wii,2006.0,Sports,Activision,0.51,0.00,0.00,0.04,0.56
1,Casual Mania!,DS,2010.0,Puzzle,Foreign Media Games,0.04,0.00,0.00,0.00,0.04
2,Atelier Sophie: The Alchemist of the Mysteriou...,PS3,2015.0,Role-Playing,Tecmo Koei,0.00,0.00,0.03,0.00,0.03
3,Nicola Kanshuu: Model * Oshare Audition,DS,2010.0,Misc,Alchemist,0.00,0.00,0.06,0.00,0.06
4,Oddworld: Munch's Oddysee,GBA,2003.0,Platform,THQ,0.04,0.02,0.00,0.00,0.06
...,...,...,...,...,...,...,...,...,...,...
16592,Playboy: The Mansion,PS2,2005.0,Simulation,Ubisoft,0.10,0.08,0.00,0.03,0.20
16593,Star Wars Episode III: Revenge of the Sith,GBA,2005.0,Action,Ubisoft,0.52,0.19,0.00,0.01,0.72
16594,Gears of War,X360,2006.0,Shooter,Microsoft Game Studios,3.54,1.90,0.07,0.60,6.11
16595,Hanasaku Manimani,PSV,2014.0,Adventure,5pb,0.00,0.00,0.01,0.00,0.01


### Sorting Dataframes

In [15]:
# sorting first by year, then by global sales
dfMB = dfMB.sort_values(by=['Year','Global_Sales'],ascending=[True, False])
# Also getting rid of any rows that dont have a year
# So we can see the full range
dfMB = dfMB.dropna(subset=['Year'])
dfMB

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
611,Asteroids,2600,1980.0,Shooter,Atari,4.00,0.26,0.00,0.05,4.31
6243,Missile Command,2600,1980.0,Shooter,Atari,2.56,0.17,0.00,0.03,2.76
10797,Kaboom!,2600,1980.0,Misc,Activision,1.07,0.07,0.00,0.01,1.15
5637,Defender,2600,1980.0,Misc,Atari,0.99,0.05,0.00,0.01,1.05
11768,Boxing,2600,1980.0,Fighting,Activision,0.72,0.04,0.00,0.01,0.77
...,...,...,...,...,...,...,...,...,...,...
16240,Dragon's Dogma Online: Season 2,PS4,2016.0,Action,Capcom,0.00,0.00,0.01,0.00,0.01
3117,Phantasy Star Online 2 Episode 4: Deluxe Package,PS4,2017.0,Role-Playing,Sega,0.00,0.00,0.03,0.00,0.03
5205,Brothers Conflict: Precious Baby,PSV,2017.0,Action,Idea Factory,0.00,0.00,0.01,0.00,0.01
9799,Phantasy Star Online 2 Episode 4: Deluxe Package,PSV,2017.0,Role-Playing,Sega,0.00,0.00,0.01,0.00,0.01


We can see it spans about 40 years with the data we have left

In [16]:
print("The first game with the most sales is: {}".format(dfMB['Name'].iloc[0]))
print("The first car is: {}".format(dfMB['Name'].loc[0]))
# check readout from shuffling frames if confused by what above line produces

#loc gets rows (or columns) with particular labels from the index.
#iloc gets rows (or columns) at particular positions in the index (so it only takes integers).

The first game with the most sales is: Asteroids
The first car is: Rapala Tournament Fishing!


### Saving a Dataframe

In [17]:
import os

# Relative path within the repository
writeFlnam = os.path.join("data", "vgsalesMB.csv")

# Ensure the 'data' directory exists
os.makedirs(os.path.dirname(writeFlnam), exist_ok=True)

# Save the DataFrame
dfMB.to_csv(writeFlnam, index=False)
print("DataFrame saved to: {}".format(writeFlnam))

DataFrame saved to: data\vgsalesMB.csv
